In [1]:
import pandas as pd
import windpy as wp

In [71]:
df = pd.read_csv('All_Results_TMYx.csv', index_col=0) 
df

,level_0,Location Name,Coordinates,WMO,last_year,Country,Latitude,Longitude,Heating DB 99.6%,Heating DB 99%,...,Heating DB 99.6% - EPW,Heating DB 99% - EPW,Cooling DB 0.4% - EPW,Cooling DB 1% - EPW,Region_number,Region,delta - Heating DB 99.6%,delta - Heating DB 99%,delta - Cooling DB 0.4%,delta - Cooling DB 1%
0,0,Capmon.CS-Saturna.Island BC CAN,"['-123.1281', '48.77530', '178.0']",719140,TMYx,Canada,48.7753N,123.1281W,-4.6,-2.9,...,-30.9640,-23.300,22.0000,21.000,71,North America,26.3640,20.400,4.7000,3.800
1,1,Brockville-Tackaberry.Thousand.Islands.Rgnl.AP...,"['-75.75160', '44.63540', '121.4']",711610,TMYx,Canada,44.6354N,75.7516W,-24.5,-21.1,...,-44.4820,-37.123,28.3964,26.700,71,North America,19.9820,16.023,1.3036,1.500
2,2,Cheticamp.CS NS CAN,"['-60.94722', '46.64500', '44.0']",719590,TMYx,Canada,46.650N,60.950W,-16.0,-13.7,...,-32.2964,-30.800,23.5964,22.100,71,North America,16.2964,17.100,3.7036,3.800
3,3,Interlaken.AP BE CHE,"['7.87030', '46.67220', '578.6']",67340,TMYx,Switzerland,46.6722N,7.8703E,-8.3,-6.6,...,-23.8964,-22.000,6.0000,5.000,67,Africa,15.5964,15.400,23.3000,22.300
4,4,Baccaro.Point NS CAN,"['-65.47170', '43.45080', '5.7']",719200,TMYx,Canada,43.4508N,65.4717W,-12.4,-10.2,...,-27.5892,-21.000,24.2000,22.341,71,North America,15.1892,10.800,-4.2000,-3.441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8851,8851,Yerevan-Erebuni.AP ER ARM,"['44.53000', '40.21200', '901.0']",377890,TMYx,Armenia,40.212N,44.530E,-13.1,-10.7,...,NaN,NaN,NaN,NaN,37,Asia,NaN,NaN,NaN,NaN
8852,8852,Amasia SH ARM,"['43.78300', '40.95000', '1866.0']",376820,TMYx,Armenia,40.950N,43.784E,-19.0,-16.7,...,NaN,NaN,NaN,NaN,37,Asia,NaN,NaN,NaN,NaN
8853,8853,Gyumri SH ARM,"['43.85600', '40.76300', '1512.0']",376860,TMYx,Armenia,40.763N,43.856E,-23.1,-20.9,...,NaN,NaN,NaN,NaN,37,Asia,NaN,NaN,NaN,NaN
8854,8854,Lake.Sevan GR ARM,"['45.00830', '40.56580', '1917.0']",377170,TMYx,Armenia,40.5653N,45.0083E,-13.5,-11.7,...,NaN,NaN,NaN,NaN,37,Asia,NaN,NaN,NaN,NaN


In [72]:
## Read in an EPW file to a Pandas dataframe
def readNOAA_DesignDay(df):
    htg_996= df['Temperature'].quantile(q=0.004)
    htg_990= df['Temperature'].quantile(q=0.010)
    clg_004= df['Temperature'].quantile(q=0.996)
    clg_010= df['Temperature'].quantile(q=0.99)
    return htg_996,htg_990,clg_004,clg_010

In [73]:
def getDDFromNOAA(lat,lon,rad=10,years=10):
    stations = wp.NOAA.findStation(centre = [lat, lon], radius = rad, unit = 'km')
    startYear =stations['BEGIN'][0].year
    endYear = stations['END'][0].year
    newStartYear  = endYear-years
    if startYear > newStartYear:
        newStartYear = startYear
    if stations.shape[0] > 0:
        id_loc = stations.index.astype('str')[0]
    else:
        id_loc = None
    if id_loc is not None:
        raw_data = wp.NOAA.getNOAA(ID = id_loc, yearStart = newStartYear, yearEnd = endYear)
        df = raw_data[raw_data.Temperature != 999.9]
    else:
        df = None
    return df

In [74]:
s_htg_996_epw= []
s_htg_990_epw= []
s_clg_004_epw= []
s_clg_010_epw= []

for index in df.index:
    lat = df.loc[index,'Latitude']
    lon = df.loc[index,'Longitude']
    dir_lat = lat[-1]
    latVal = float(lat[0:len(lat)-1])
    dir_lon = lon[-1]
    lonVal = float(lon[0:len(lon)-1])
    if dir_lat == 'S':
        latValDir = latVal*-1
    else:
        latValDir=latVal
    if dir_lon == 'W':
        lonValDir= lonVal*-1
    else:
        lonValDir = lonVal
    #print(latValDir,lonValDir)
    df_noaa = getDDFromNOAA(latValDir,lonValDir)
    htg_996,htg_990,clg_004,clg_010 = readNOAA_DesignDay(df_noaa)
    #print(htg_996,htg_990,clg_004,clg_010)
    s_htg_996_epw.append(htg_996)
    s_htg_990_epw.append(htg_990)
    s_clg_004_epw.append(clg_004)
    s_clg_010_epw.append(clg_010)

df['Heating DB 99.6% - NOAA'] = s_htg_996_epw
df['Heating DB 99% - NOAA'] = s_htg_990_epw
df['Cooling DB 0.4% - NOAA'] = s_clg_004_epw
df['Cooling DB 1% - NOAA'] = s_clg_010_epw
filename = 'All_Results_TMYx_NOAA.csv'
export_csv = df.to_csv(filename, index = None, header=True) 

Loading info local backup from C:\ProgramData\Anaconda3\lib\site-packages\windpy-0.3.0-py3.9.egg\windpy\NOAA\tempdata\isd-history.txt
Loading local backup from C:\ProgramData\Anaconda3\lib\site-packages\windpy-0.3.0-py3.9.egg\windpy\NOAA\tempdata\71473099999_2012_2022.pkl
Loading info local backup from C:\ProgramData\Anaconda3\lib\site-packages\windpy-0.3.0-py3.9.egg\windpy\NOAA\tempdata\isd-history.txt
Loading local backup from C:\ProgramData\Anaconda3\lib\site-packages\windpy-0.3.0-py3.9.egg\windpy\NOAA\tempdata\71161099999_2012_2022.pkl
Loading info local backup from C:\ProgramData\Anaconda3\lib\site-packages\windpy-0.3.0-py3.9.egg\windpy\NOAA\tempdata\isd-history.txt
Loading local backup from C:\ProgramData\Anaconda3\lib\site-packages\windpy-0.3.0-py3.9.egg\windpy\NOAA\tempdata\71959026394_1983_1993.pkl
Loading info local backup from C:\ProgramData\Anaconda3\lib\site-packages\windpy-0.3.0-py3.9.egg\windpy\NOAA\tempdata\isd-history.txt
Loading local backup from C:\ProgramData\Anacon

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))